In [1]:
from utils import MDFeat,load,save
import pandas as pd
import numpy as np
from utils import MDFeat
from typing import List
from transformers import pipeline,AutoModelForTokenClassification,AutoTokenizer
from tqdm import tqdm

In [14]:
data:List[MDFeat] = load("/home/tywang/MD/data/VUA/vua_train.pkl")

In [15]:
data[0]

MDFeat(sentence="Ca n't fail to be entertaining .", verb_idx=2, verb='fail', label=0, addinfo1=["Ca n't fails to be entertaining .", "Ca n't failure to be entertaining .", "Ca n't failed to be entertaining .", "Ca n't succeed to be entertaining .", "Ca n't failing to be entertaining .", "Ca n't try to be entertaining .", "Ca n't attempt to be entertaining .", "Ca n't have to be entertaining .", "Ca n't cease to be entertaining ."], addinfo2=None)

In [11]:
def get_mask_sentence(unmasker,data:MDFeat):
    unmasked_texts = []
    sentence_template = data.sentence.split()
    ori_token = sentence_template[data.verb_idx]
    sentence_template[data.verb_idx] = '{}'
    sentence_template = " ".join(sentence_template)
    input_sentence = sentence_template + " " + data.sentence
    input_sentence = input_sentence.format("[MASK]")
    unmask_result = unmasker(input_sentence)
    for result in unmask_result:
        if result['token_str'].find():
            unmasked_texts.append(sentence_template.format("<VERB> "+result['token_str']+" </VERB>"))
    return unmasked_texts

In [16]:
unmasker = pipeline('fill-mask', model='/data/transformers/bert-base-uncased',device = 1,top_k =10)

Some weights of the model checkpoint at /data/transformers/bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
for i in tqdm(range(len(data))):
    data[i].addinfo1 = get_mask_sentence(unmasker,data[i])

100%|██████████| 15516/15516 [03:55<00:00, 65.78it/s]


In [19]:
save(data,"/home/tywang/MD/data/VUA/vua_train.pkl")

save to(保存到)  /home/tywang/MD/data/VUA/vua_train.pkl


In [20]:
data[0]

MDFeat(sentence="Ca n't fail to be entertaining .", verb_idx=2, verb='fail', label=0, addinfo1=["Ca n't <VERB> fails </VERB> to be entertaining .", "Ca n't <VERB> failure </VERB> to be entertaining .", "Ca n't <VERB> failed </VERB> to be entertaining .", "Ca n't <VERB> succeed </VERB> to be entertaining .", "Ca n't <VERB> failing </VERB> to be entertaining .", "Ca n't <VERB> try </VERB> to be entertaining .", "Ca n't <VERB> attempt </VERB> to be entertaining .", "Ca n't <VERB> have </VERB> to be entertaining .", "Ca n't <VERB> cease </VERB> to be entertaining ."], addinfo2=None)

In [13]:
def get_mask_sentence(unmasker,text_arr:list):
    """
        输入一个句子arr, 得到它的预测token以及预测的score
    """
    unmask_result = unmasker(text_arr)
    return_results = []
    for result in unmask_result:
        return_result = []
        for idx,token in enumerate(result):
            return_result.append((token['score'],token['token_str']))
        return_results.append(return_result)
    return return_results
    

In [14]:
get_mask_sentence(unmasker,["Hello I'm a [MASK] model.","Hello I'm a [MASK] model."])

[[(0.10721077024936676, 'fashion'),
  (0.08796188235282898, 'role'),
  (0.05333735793828964, 'new'),
  (0.046707648783922195, 'super'),
  (0.027107620611786842, 'fine'),
  (0.02393934689462185, 'good'),
  (0.021875906735658646, 'model'),
  (0.0213792622089386, 'great'),
  (0.01942354626953602, 'business'),
  (0.016654005274176598, 'fitness')],
 [(0.10721077024936676, 'fashion'),
  (0.08796188235282898, 'role'),
  (0.05333735793828964, 'new'),
  (0.046707648783922195, 'super'),
  (0.027107620611786842, 'fine'),
  (0.02393934689462185, 'good'),
  (0.021875906735658646, 'model'),
  (0.0213792622089386, 'great'),
  (0.01942354626953602, 'business'),
  (0.016654005274176598, 'fitness')]]

In [14]:
mode = "test"
df = pd.read_csv("/home/tywang/MD/data/VUA/VUA_formatted_{}.csv".format(mode), encoding = "ISO-8859-1",low_memory=False)

In [15]:
df.head()

,text_idx,sentence_idx,verb,sentence,verb_idx,label
0,a3m-fragment02,45,cross,Design : Crossed lines over the toytown tram :...,2,1
1,a3m-fragment02,45,say,Design : Crossed lines over the toytown tram :...,20,0
2,a3m-fragment02,47,know,"MODERN trams , as most continental Europeans k...",7,0
3,a3m-fragment02,47,shakdevoure,"MODERN trams , as most continental Europeans k...",10,0
4,a3m-fragment02,47,rattle,"MODERN trams , as most continental Europeans k...",12,0


In [16]:
sentence = df['sentence'].values
verb_idx = df['verb_idx'].values
verb = df['verb'].values
label = df['label'].values
senetnce = [item.strip() for item in sentence]

In [17]:
mohx = list()
for text, verb_id, v, gt in zip(senetnce,verb_idx,verb,label):
    feat = MDFeat(
        sentence=text,
        verb_idx = verb_id,
        verb = v,
        label = gt,
        addinfo1=None,
        addinfo2=None
    )
    mohx.append(feat)

In [18]:
mohx[-10:]

[MDFeat(sentence='The trouble with all the views which we have looked at is that they tie the notion of autonomy firmly and solely to that of knowledge interpreted in either a broad or narrow sense .', verb_idx=9, verb='look', label=1, addinfo1=None, addinfo2=None),
 MDFeat(sentence='The trouble with all the views which we have looked at is that they tie the notion of autonomy firmly and solely to that of knowledge interpreted in either a broad or narrow sense .', verb_idx=14, verb='tie', label=1, addinfo1=None, addinfo2=None),
 MDFeat(sentence='The trouble with all the views which we have looked at is that they tie the notion of autonomy firmly and solely to that of knowledge interpreted in either a broad or narrow sense .', verb_idx=26, verb='interpret', label=1, addinfo1=None, addinfo2=None),
 MDFeat(sentence='That it is firmly tied must be correct ; the inhabitants of Brave New World are not autonomous precisely because they are denied access to relevant information .', verb_idx=4,

In [19]:
save(mohx,"/home/tywang/MD/data/VUA/vua_{}.pkl".format(mode))

save to(保存到)  /home/tywang/MD/data/VUA/vua_test.pkl
